In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm_notebook as tqdm
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=0)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=0)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
from trainer.base_trainer import BaseTrainer
from logger.new_callbacks import Callbacks

class Trainer(BaseTrainer):
    def __init__(self):
        
        self.trainloader = trainloader
        self.validloader = testloader
        self.cb = Callbacks()
        
        self.criterion = nn.CrossEntropyLoss()
        
        self.lr = 0.0025
        
        self.init_model()
        self.init_optimizer()
        self.init_scheduler()
        
        
        self.use_amp = False
        self.grad_acc_num = 1
        
        self.epochs = 2
    
    def init_model(self):
        self.model = Net()
    
    def init_optimizer(self, lr=None):
        if lr is not None:
            self.lr = lr
        self.optimizer = optim.SGD(self.model.parameters(), lr=self.lr, momentum=0.9)
    
    def init_scheduler(self):
        self.scheduler = None
    '''
    1.1.1. batch process
    '''
    def batch_process(self, batch, index=None, isTraining=True):
        self.cb.on_batch_process_start()

        self.cb.on_batch_process_end()
        return batch

    '''
    1.1.2. batch train
    '''
    def batch_train_step(self, batch, index):
        self.cb.on_batch_train_step_start()
        
        inputs, labels = batch
        outputs = self.model(inputs)
        loss = self.criterion(outputs, labels)
        
        
        dict_metrics = {"train_batch_loss":loss.item()}
        if self.scheduler is not None:
            dict_metrics["lr"] = self.scheduler.get_lr()[0]

        self.cb.on_batch_train_step_end(dict_metrics)
        return loss
    
    '''
    2.1.2. batch valid
    '''
    def batch_valid_step(self, batch, index):
        self.cb.on_batch_valid_step_start()
        
        inputs, labels = batch
        outputs = self.model(inputs)
        loss = self.criterion(outputs, labels)

        self.cb.on_batch_valid_step_end({"valid_batch_loss":loss.item()})

In [ ]:
trainer = Trainer()

In [ ]:
# trainer.train_on_sample(steps=1000)

In [ ]:
trainer.train()

In [ ]:
num_iter = 10000
start_lr = 0.0001
end_lr = 1.0
step_mode = "exp"
stop_factor = 5
lrs, losses, min_lr_index = trainer.lr_finder(num_iter, start_lr, end_lr, step_mode, stop_factor)

In [ ]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
df = pd.DataFrame({"loss":losses, "lr":lrs})
df = df[0:min_lr_index+100]
df = df.set_index("lr")
plt.plot(df["loss"], 'lightblue', df["loss"].rolling(100).mean(), 'b');
plt.ylim(1.5,2.9)